<a href="https://colab.research.google.com/github/salmaahmedtaha/CodeAlpha_Spotify_recommender/blob/main/spotify_recommender1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/spotify_recommender

/content/drive/MyDrive/spotify_recommender


In [4]:
!mkdir ~/.kaggle/
! cp /content/drive/MyDrive/spotify_recommender/kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d notshrirang/spotify-million-song-dataset

Dataset URL: https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset
License(s): CC0-1.0
spotify-million-song-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("/content/drive/MyDrive/spotify_recommender/spotify_millsongdata.csv")

In [9]:

df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [10]:

df.shape

(57650, 4)

In [11]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [12]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [13]:
df.head(10)

,artist,song,text
0,Bon Jovi,Diamond Ring,"Diamond ring, wear it on your hand \r\nIt's g..."
1,System Of A Down,Innervision,I have a home \r\nLonging to roam \r\nI have...
2,Europe,Reason,And everybody will get to play \r\nThe hand t...
3,Bread,She Was My Lady,She Was My Lady \r\n(David Gates) \r\n \r\n...
4,Glee,Defying Gravity,Something has changed within me \r\nSomething...
5,Sam Smith,Restart,It was a Monday night when you told me it was ...
6,George Strait,Overnight Male,Let me be your male-man \r\nAnd I'll always c...
7,Hollies,Gasoline Alley Bred,Oh woman get your head out of curlers ! \r\nT...
8,Ian Hunter,The Artful Dodger,"How ya doin', all right? \r\nI-I-I spy with m..."
9,Nick Cave,The Mercy Seat,It began when they come took me from my home ...


In [14]:
df['text'][0]

"Diamond ring, wear it on your hand  \r\nIt's gonna tell the world, I'm your only man  \r\nDiamond ring, diamond ring  \r\nBaby, you're my everything, diamond ring  \r\n  \r\nRed, red rose brought it home to you  \r\nBlood red rose, tells me that you're true  \r\nRed, red rose, blood-red rose  \r\nLike a fire inside that grows, blood-red rose  \r\n  \r\nWhen you're hungry, I will fill you up  \r\nWhen you're thristy, drink out of my loving cup  \r\nWhen you're crying, I'll be the tears for you  \r\nThere's nothing that I wouldn't do for you  \r\n  \r\nWhen you're hungry, I will fill you up  \r\nWhen you're thristy, drink out of my loving cup  \r\nWhen you're crying, I'll be the tears for you  \r\nThere's nothing that I wouldn't do for you  \r\n  \r\nYou know, I bleed every night you sleep  \r\n'Cause I don't know if I'm in your dreams  \r\nI want to be your everything...  \r\n  \r\nDiamond ring, wear it on your hand  \r\nIt's gonna tell the world, I'm your only man  \r\nDiamond ring, d

In [15]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [19]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:

tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [21]:

similarity[0]

array([1.        , 0.00105458, 0.00945361, ..., 0.01611385, 0.01076443,
       0.00697517])

In [25]:
df[df['song'] == 'Diamond Ring']

,artist,song,text
0,Bon Jovi,Diamond Ring,"diamond ring, wear it on your hand \r it's go..."


In [23]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [26]:
recommendation('Diamond Ring')

['Mockingbird',
 'Let It Ring',
 'Jack-A-Diamonds',
 'Love Is A Golden Ring',
 'Ring Them Bells',
 'Red (In My) Head',
 'Plain Gold Ring',
 "Can't Buy Me Love",
 'Silver Bells',
 'Silver Bells',
 'Nocturn',
 "I Don't Want Your Money",
 'Mood Ring',
 'Bring It All To Me',
 'Ballad Of The Boy In The Red Shoes',
 'Red',
 'Red Rose',
 "I'm On Fire",
 'Earthbound Gypsy',
 "I Can't Give You Anything But Love"]

In [28]:
recommendation('She Was My Lady')

['My Sweet Lady',
 'Sanctuary',
 'Hello Lady Goodbye',
 'Lady Godiva',
 'Lover In Disguise',
 'Walks Like A Lady',
 'Golden Lady',
 'If You Knew What I Knew',
 'Lover Lay Down',
 'Lady',
 "Lady D'arbanville",
 'Sanctuary',
 'Could You Be Loved',
 'I Loved Another Woman',
 'Jesus To A Child',
 'Did You Not Hear My Lady',
 'Queen For A Day',
 'April Lady',
 'Ukulele Lady',
 'Lay Me Down']

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))